# Quick and dirty test of rough draft of SOMPZInformer

In [1]:
# usual imports
import os
import numpy as np
from rail.core.utils import RAILDIR
#from rail.estimation.algos.sompz_version.utils import RAIL_SOMPZ_DIR
#from rail.pipelines.estimation.inform_all import InformPipeline
from rail.core import common_params?
import matplotlib.pyplot as plt

Object `common_params` not found.


In [2]:
from rail.estimation.algos.sompz import SOMPZInformer

In [3]:
from rail.core.data import TableHandle
from rail.core.stage import RailStage

In [4]:
DS = RailStage.data_store
DS.__class__.allow_overwrite = True

read in some test data for the "wide" som, it's just 20,000 mock ugrizy magnitudes and errors from cosmodc2

In [5]:
from rail.core.utils import find_rail_file
# trainFile = find_rail_file('examples_data/testdata/test_dc2_training_9816.hdf5')
# testFile = find_rail_file('examples_data/testdata/test_dc2_validation_9816.hdf5')
# deep_data = DS.read_file("training_data", TableHandle, trainFile)
# wide_data = DS.read_file("input_wide_data", TableHandle, testFile)

I have copied the file that Justin made to this directory and will read that fits file in to test the "deep" som.  If you have the file somewhere else, then update the "hsc_file" path in the cell below:

In [6]:
#datadir = '/global/cfs/projectdirs/desc-pz/users/jmyles/sompz_desc/' 
datadir = '/Users/jmyles/data/sompz_desc'
hsc_file = os.path.join(datadir, "hsc_deep_incl_nir_incl_specz_TEST_ONLY_NO_SCIENCE_2023-10-27.fits")
deep_data = DS.read_file("training_data", TableHandle, hsc_file)
wide_data = DS.read_file("training_data", TableHandle, hsc_file)

In [8]:
#deep_data.data.colnames#['flux_DES_DEEP_u']

To speed things up a bit, I'm going to grab 10% of the data and actually use that.  It should work with the big data, but this will be faster to train up the som for a quick demo:

In [9]:
subset = deep_data()[::10]

In [10]:
smaller_data = DS.add_data("smaller_data", subset, TableHandle)

create a few lists of names that we'll need for the names of the input columns that we'll used in the SOM, along with their errors.

In [15]:
bands = ['u','g','r','i','z','y','J','H', 'K']

deepbands = []
deeperrs = []
zeropts = []
for band in bands:
    deepbands.append(f'flux_DEEP_{band}')
    deeperrs.append(f'flux_err_DEEP_{band}')
    zeropts.append(30.)

widebands = []
wideerrs = []  
for band in bands[1:5]:
    widebands.append(f'{band}_cmodel_flux')
    wideerrs.append(f'{band}_cmodel_fluxerr')
    
refband_deep=deepbands[3]
refband_wide=widebands[2]

make a dictionary of the parameters that we'll feed into the informer for the deep som.  Because we're using HSC data we need the non-default names fo the input columns (`inputs_deep`) and the errors (`input_errs_deep`).  We'll feed in a list for the zero points (`zero_points`) as well, as eventually we'll want to add a check to make sure all three of those have the same length:

In [16]:
som_params = dict(inputs_deep=deepbands, input_errs_deep=deeperrs, zero_points_deep=zeropts, 
                  inputs_wide=widebands, input_errs_wide=wideerrs, #zero_points_deep=zeropts, 
                  convert_to_flux_deep=False, convert_to_flux_wide=False, 
                  set_threshold_deep=True, thresh_val_deep=1.e-5, 
                  som_shape_wide=(25, 25), som_minerror_wide=0.005,
                  som_take_log_wide=False, som_wrap_wide=False)

deep_groupname and hdf5_groupname (along with an unused wide_groupname) have to do with hdf5 groups and how we read the data, hdf5_groupname is not used in this stage, but it is expected by the parent class, so I'm setting it here:

Note that we did not set the som_shape_deep, so it will default to (32, 32), but we did set the shape for the wide to (25, 25):

In [17]:
som_inform = SOMPZInformer.make_stage(name="som_informer", 
                                      deep_groupname='', 
                                      hdf5_groupname='',
                                      wide_groupname='',
                                      model="TEST_HSC_model_2023-10-27b.pkl", **som_params)

Now, run the informer:

In [18]:
som_inform.inform(smaller_data, smaller_data)

3546 galaxies in deep sample
3546 galaxies in wide sample
Training deep SOM of shape (32, 32)...
Training 0
Training wide SOM of shape (25, 25)...
Training 0


/Users/jmyles/repositories/rail_sompz/src/rail/estimation/algos/som.py:360: RuntimeWarning: overflow encountered in exp
  w = np.minimum(np.exp(2 * (vf - 4)), 1000.)


Inserting handle into data store.  model_som_informer: inprogress_TEST_HSC_model_2023-10-27b.pkl, som_informer


{'deep_som': <rail.estimation.algos.som.NoiseSOM at 0x16d1ab4f0>,
 'wide_som': <rail.estimation.algos.som.NoiseSOM at 0x16d1ab5e0>,
 'deep_columns': ['flux_DEEP_u',
  'flux_DEEP_g',
  'flux_DEEP_r',
  'flux_DEEP_i',
  'flux_DEEP_z',
  'flux_DEEP_y',
  'flux_DEEP_J',
  'flux_DEEP_H',
  'flux_DEEP_K'],
 'deep_err_columns': ['flux_err_DEEP_u',
  'flux_err_DEEP_g',
  'flux_err_DEEP_r',
  'flux_err_DEEP_i',
  'flux_err_DEEP_z',
  'flux_err_DEEP_y',
  'flux_err_DEEP_J',
  'flux_err_DEEP_H',
  'flux_err_DEEP_K'],
 'wide_columns': ['g_cmodel_flux',
  'r_cmodel_flux',
  'i_cmodel_flux',
  'z_cmodel_flux'],
 'wide_err_columns': ['g_cmodel_fluxerr',
  'r_cmodel_fluxerr',
  'i_cmodel_fluxerr',
  'z_cmodel_fluxerr']}

In [22]:
np.sum(~np.isfinite(som_inform.model['deep_som'].weights))

0

In [27]:
som_inform.model['deep_som'].shape[0]

32

In [23]:
np.sum(~np.isfinite(som_inform.model['wide_som'].weights))

0

Let's look at the results by reading the model we just wrote out, `TEST_HSC_DC2_model.pkl` and looking at what it contains.  It should be a dictionary that contains two soms: `deep_som` and `wide_som`, along with the `deep_columns`, `wide_columns`, `deep_err_columns`, and `wide_err_columns` lists.  I think we'll want to store these as they basically define the ordering of the columns and errors, and we'll want that the same for data that we pass in (if we pass in more data?  I need to look at how the estimate stage works...)

In [ ]:
import pickle

In [ ]:
with open("TEST_HSC_model_2023-10-27.pkl", "rb") as f:
    model = pickle.load(f)

In [ ]:
model

In [ ]:
import rail.estimation.algos.som as SOMFUNCS

In [ ]:
SOMFUNCS.somDomainColorsnok(model['deep_som'])

In [ ]:
SOMFUNCS.somDomainColors(model['deep_som'])

In [ ]:
SOMFUNCS.somDomainColorsnok(model['wide_som'])

In [ ]:
SOMFUNCS.somPlot2d(model['deep_som'])